# Hiperparaméterek optimalizációja

A paraméteroptimalizációhoz a Bayesi optimalizációs technikát alkalmazom. Ehhez meg kell hívni a szükséges könyvtárat.

In [ ]:
#uncomment the next line if the package is not yet installed in your machine
#!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
import sys
sys.path.append('c:\\users\\ifjto\\appdata\\local\\programs\\python\\python37\\lib\\site-packages')
import numpy as np
import import_ipynb
from model import *
import torch
from torch import nn
from torch.utils.data import *
import torch.optim as optim
import torch.cuda
import torchvision.transforms as transforms
import cv2
import os
import time
from IPython.display import Image
import math

Az optimalizálandó paraméterek értékkészletét állítsuk be. Természetesen nem megengedhetőek az óriási értékek, mivel nincs rá elegendő számítási kapacitás - áldozatot kell hoznunk. Ezen persze segíthetünk azzal, hogy több kicsi optimalizálást futtatunk, és megpróbáljuk kitalálni, hogy melyik paraméterértékek túl kicsik vagy nagyok, így szűkítve az optimalizáló értékkészletét. Néhány magyarázat a választott határokhoz:

In [ ]:
# the maximum number of pictures included in each epoch
TRAIN_SIZE = 5000

pbounds = {'nFeat' : (16, 128),
           'dropout' : (0.2, 0.8),
           'auto_lr' : (-4.1, -2.5),
           'disc_lr' : (-7, -3),
           'ratio' : (0.25, 0.8),
           'steps_without_disc' : (3, 8),
           'decay' : (-9, -3)}

Olvassuk be az adatokat egyszer, hogy ne kelljen minden hiperparaméterkombinációra megint.

In [ ]:
trainloader, testloader, validationloader, discriminator_dataset, disc_loader = get_data(32, TRAIN_SIZE)

Készítsünk egy függvényt, amit az optimalizálónk tud használni. A függvény felelőssége legyen, hogy csak elfogadható értékekkel híva a train-t.

In [ ]:
def test_parameters(auto_lr, disc_lr, ratio, dropout, steps_without_disc, decay, nFeat):
    global trainloader, testloader, discriminator_dataset, disc_loader
    NUM_EPOCH = 20
    auto_lr = pow(10, auto_lr)
    disc_lr = pow(10, disc_lr)
    steps_without_disc = int(round(steps_without_disc))
    decay = pow(10, decay)
    nFeat = int(round(nFeat))
    
    # redirecting stdout to avoid disturbing prints
    original_out = sys.stdout
    sys.stdout = open("logs.txt", "a")
    
    # building network
    autoencoder = Autoencoder(nFeat)
    auto_optimizer = optim.Adam(autoencoder.parameters(), lr=auto_lr)
    auto_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(auto_optimizer, NUM_EPOCH)
    
    discriminator = Discriminator(dropout)
    disc_optimizer = optim.Adam(discriminator.parameters(), lr=disc_lr)
    disc_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(disc_optimizer, NUM_EPOCH)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        autoencoder = autoencoder.cuda()
        discriminator = discriminator.cuda()
    
    # training network
    train(autoencoder, discriminator, trainloader, discriminator_dataset, disc_loader, NUM_EPOCH, ratio=ratio, 
          steps_without_disc=steps_without_disc, TRAIN_SIZE=TRAIN_SIZE, AUTO_LR=auto_lr, DISC_LR=disc_lr, wd=decay)
    
    r1, r2 = get_result(autoencoder, testloader)
    result = r1*0.1 + r2*0.9
    
    # saving results
    sys.stdout = open("results.csv", "a")
    print(str(result)+","+str(r1)+","+str(r2)+","+str(math.log(auto_lr, 10))+","+str(math.log(disc_lr, 10))+","+
          str(ratio)+","+str(dropout)+","+str(steps_without_disc)+","+str(math.log(decay, 10))+","+str(nFeat))
    # resetting the stdout
    sys.stdout = original_out
    return result

Optimalizáljunk. A jó eredményhez kezdjük néhány random lépéssel a próbálkozást.

In [ ]:
start_time = time.time()
optimizer = BayesianOptimization(
    f=test_parameters,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=25,
    n_iter=75,
)

print(optimizer.max)
print('Finished after ', round(time.time() - start_time), " seconds")

Az értékkészletek csökkentéséhez rajzoljuk ki az eredmények függését az egyes hiperparaméterektől. Természetesen elképzelhető sokkal komplexebb összefüggés is a hiperparaméterek között, azonban ennyi paraméter együttes vizualizációja óriási kihívás lenne.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib widget

#reading file
lines = []
with open("results.csv", "r") as file:
    lines = file.readlines()
#preparing input
header = lines[0][:-1].split(",")
del(lines[0])

columns = [ [] for i in range(len(header)) ]
for i in range(len(lines)):
    for k in range(len(header)):
        columns[k].append( float(lines[i][:-1].split(",")[k]) )

for i in range(3, len(header)):
    f = plt.figure(figsize=(7, 4))
    plt.ylabel('reality_score')
    plt.xlabel(header[i])
    plt.scatter(columns[i], columns[0], s=50, alpha=0.25, edgecolors='none')
    f.show()